## Make requests with comma separated params

In [2]:
import requests

In [3]:
def get_request_with_comma_separated_params(base_url, resource, param_name, values):
    params = {param_name: ','.join(map(str, values))}
    response = requests.get(f'{base_url}/{resource}', params=params)
    return response

# Usage
float_values = [1.1, 2.2, 3.3]
response = get_request_with_comma_separated_params('http://example.com', 'api/resource', 'values', float_values)
print(response.url)

http://example.com/api/resource?values=1.1%2C2.2%2C3.3


In [4]:
print(response)

<Response [404]>


## Test the associated GET request

In [52]:
def get_model_from_api():
    url = 'http://127.0.0.1:5000/api/VIC'
    response = requests.get(url)
    if response.status_code == 200:
        print('Talked to API, Success')
        return response.json()
    else:
        return {'error': 'Failed to retrieve values from API'}

In [53]:
model = get_model_from_api()
model

Talked to API, Success


{'coef': [1.0, 1.9999999999999993], 'intercept': 3.0000000000000018}

In [41]:
def get_prediction_from_api(values):
    url = 'http://127.0.0.1:5000/api/VIC'
    params = {'predictors': ','.join(map(str, values))}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        print('Talked to API, Success')
        return response.json()
    else:
        return {'error': 'Failed to retrieve values from API'}

In [42]:
values = [1.1, 2.2]
result = get_prediction_from_api(values)
print(result)

Talked to API, Success
{'prediction': 8.5}


In [51]:
values = [1.1, 2.2, 3.3]
result = get_prediction_from_api(values)
print(result)

Talked to API, Success
{'Status': 400, 'Message': 'Invalid Parameters'}


## Generate example model

In [43]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pickle

In [44]:
# Generate some sample data
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
y = np.dot(X, np.array([1, 2])) + 3

# Train the linear regression model
model = LinearRegression().fit(X, y)

# Print the coefficients
print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')

Coefficients: [1. 2.]
Intercept: 3.0000000000000018


In [45]:
with open('lin_reg_model_test.pkl', 'wb') as file:
    pickle.dump(model, file)